In [1]:
path_model = 'C:/Users/sOrOush/SoroushProjects/02_Linearity_Project/02_Models/vdvae'

import sys
sys.path.append(path_model)
# Check if the directory has been added
print("Current sys.path:", sys.path)

# Attempt to import the module
try:
    from model_utils import *
    print("Module model_utils successfully imported.")
except ImportError as e:
    print(f"Error importing the module: {e}")


Current sys.path: ['C:\\Users\\sOrOush\\SoroushProjects\\14_CLIP_Ozcelic', 'C:\\Program Files\\JetBrains\\PyCharm 2024.3.2\\plugins\\python-ce\\helpers\\pydev', 'C:\\Program Files\\JetBrains\\PyCharm 2024.3.2\\plugins\\python\\helpers-pro\\jupyter_debug', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\python312.zip', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\DLLs', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question', '', 'C:\\Users\\sOrOush\\AppData\\Roaming\\Python\\Python312\\site-packages', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages\\win32', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages\\setupto

In [2]:

import torch
import numpy as np
#from mpi4py import MPI
import socket
import argparse
import os
import json
import subprocess
from hps import Hyperparams, parse_args_and_update_hparams, add_vae_arguments
from utils import (logger,
                   local_mpi_rank,
                   mpi_size,
                   maybe_download,
                   mpi_rank)
from data import mkdir_p
from contextlib import contextmanager
import torch.distributed as dist
#from apex.optimizers import FusedAdam as AdamW
from vae import VAE
from torch.nn.parallel.distributed import DistributedDataParallel
from train_helpers import restore_params
from image_utils import *
from model_utils import *
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as T
import pickle

import argparse

In [3]:
# Check if the script is running in a Jupyter Notebook
if 'ipykernel' in sys.modules:
    # Default values for Jupyter Notebook
    sub = 1  # You can specify the desired default value here
    batch_size = 30  # Default value for batch size
else:
    # Argument parser for command-line execution
    parser = argparse.ArgumentParser(description='Argument Parser')
    parser.add_argument("-sub", "--sub", help="Subject Number", default=7)
    parser.add_argument("-bs", "--bs", help="Batch Size", default=30)
    args = parser.parse_args()

    # Set the subject number and ensure it's valid
    sub = int(args.sub)
    assert sub in [1, 2, 5, 7]  # Validate allowed subject numbers

    # Set the batch size
    batch_size = int(args.bs)

# Print status messages
print('Libraries imported')
print(f'Using Subject Number: {sub}')
print(f'Using Batch Size: {batch_size}')


Libraries imported
Using Subject Number: 1
Using Batch Size: 30


In [4]:
ref_latent = np.load(r"C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\extracted_fetures_31_Layered\subj01\ref_latents_31l.npz".format(sub), allow_pickle=True)
ref_latent = ref_latent["ref_latent"]

In [5]:
import numpy as np
import torch
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as T

# Define the configuration dictionary
H = {
    'image_size': 64, 'image_channels': 3, 'seed': 0, 'port': 29500, 'save_dir': './saved_models/test',
    'data_root': './', 'desc': 'test', 'hparam_sets': 'imagenet64',
    'restore_path': r'C:\Users\sOrOush\SoroushProjects\02_Linearity_Project\02_Models\vdvae\model\imagenet64-iter-1600000-model.th',
    'restore_ema_path': r'C:\Users\sOrOush\SoroushProjects\02_Linearity_Project\02_Models\vdvae\model\imagenet64-iter-1600000-model-ema.th',
    'restore_log_path': r'C:\Users\sOrOush\SoroushProjects\02_Linearity_Project\02_Models\vdvae\model\imagenet64-iter-1600000-log.jsonl',
    'restore_optimizer_path': r'C:\Users\sOrOush\SoroushProjects\02_Linearity_Project\02_Models\vdvae\model\imagenet64-iter-1600000-opt.th',
    'dataset': 'imagenet64', 'ema_rate': 0.999, 'enc_blocks': '64x11,64d2,32x20,32d2,16x9,16d2,8x8,8d2,4x7,4d4,1x5',
    'dec_blocks': '1x2,4m1,4x3,8m4,8x7,16m8,16x15,32m16,32x31,64m32,64x12', 'zdim': 16, 'width': 512,
    'custom_width_str': '', 'bottleneck_multiple': 0.25, 'no_bias_above': 64, 'scale_encblock': False,
    'test_eval': True, 'warmup_iters': 100, 'num_mixtures': 10, 'grad_clip': 220.0, 'skip_threshold': 380.0,
    'lr': 0.00015, 'lr_prior': 0.00015, 'wd': 0.01, 'wd_prior': 0.0, 'num_epochs': 10000, 'n_batch': 4,
    'adam_beta1': 0.9, 'adam_beta2': 0.9, 'temperature': 1.0, 'iters_per_ckpt': 25000, 'iters_per_print': 1000,
    'iters_per_save': 10000, 'iters_per_images': 10000, 'epochs_per_eval': 1, 'epochs_per_probe': None,
    'epochs_per_eval_save': 1, 'num_images_visualize': 8, 'num_variables_visualize': 6, 'num_temperatures_visualize': 3,
    'mpi_size': 1, 'local_rank': 0, 'rank': 0, 'logdir': './saved_models/test/log'
}

# Dot notation for dictionary
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

H = dotdict(H)

# Assuming `set_up_data` and `load_vaes` functions are defined elsewhere
H, preprocess_fn = set_up_data(H)

print('Model is Loading')
ema_vae = load_vaes(H)

# Dataset class for external images
class batch_generator_external_images(Dataset):
    def __init__(self, data_path):
        self.data_path = data_path
        self.im = np.load(data_path).astype(np.uint8)

    def __getitem__(self, idx):
        img = Image.fromarray(self.im[idx])
        img = T.functional.resize(img, (64, 64))
        img = torch.tensor(np.array(img)).float()
        return img

    def __len__(self):
        return len(self.im)


Model is Loading
Restoring ema vae from C:\Users\sOrOush\SoroushProjects\02_Linearity_Project\02_Models\vdvae\model\imagenet64-iter-1600000-model-ema.th


C:\Users/sOrOush/SoroushProjects/02_Linearity_Project/02_Models/vdvae\train_helpers.py:126: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(distributed

In [61]:
# Transfor latents from flattened representation to hierarchical
def latent_transformation(latents, ref):
  layer_dims = np.array([2**4,2**4,2**8,2**8,2**8,2**8,2**10,2**10,2**10,2**10,2**10,2**10,2**10,2**10,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**12,2**14])
  transformed_latents = []
  for i in range(31):
    t_lat = latents[:,layer_dims[:i].sum():layer_dims[:i+1].sum()]
    #std_norm_test_latent = (t_lat - np.mean(t_lat,axis=0)) / np.std(t_lat,axis=0)
    #renorm_test_latent = std_norm_test_latent * np.std(kamitani_latents[i][num_test:].reshape(num_train,-1),axis=0) + np.mean(kamitani_latents[i][num_test:].reshape(num_train,-1),axis=0)
    c,h,w=ref[i]['z'].shape[1:]
    transformed_latents.append(t_lat.reshape(len(latents),c,h,w))
  return transformed_latents

# idx = range(len(test_images))
# input_latent = latent_transformation(pred_latents[idx],ref_latent)


def sample_from_hier_latents(latents,sample_ids):
  sample_ids = [id for id in sample_ids if id<len(latents[0])]
  layers_num=len(latents)
  sample_latents = []
  for i in range(layers_num):
    sample_latents.append(torch.tensor(latents[i][sample_ids]).float().cuda())
  return sample_latents


In [95]:
pred_latents = np.load(r"C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\polynomial_regression_results\degree4\layer_31_regression.npz")

In [96]:
pred_latents['predicted_test_latents'].shape

(982, 91168)

In [97]:
pred_latents = pred_latents['predicted_test_latents']

In [98]:
idx = range(len(pred_latents))
input_latent = latent_transformation(pred_latents[idx],ref_latent)


In [99]:
idx

range(0, 982)

In [100]:
# Verzeichnis zum Speichern der generierten Bilder
save_directory = 'C:/Users/sOrOush/SoroushProjects/14_CLIP_Ozcelic/results/generated images/VDVAE/Default/degree4/subj{:02d}'.format(sub) # no theta



In [101]:
from tqdm.notebook import tqdm

In [102]:
#pip install ipywidgets --upgrade

In [103]:
# Erstellen des Verzeichnisses, falls es nicht existiert
os.makedirs(save_directory, exist_ok=True)

for i in tqdm(range(int(np.ceil(len(pred_latents) / batch_size)))):
    print(i * batch_size)
    samp = sample_from_hier_latents(input_latent, range(i * batch_size, (i + 1) * batch_size))
    px_z = ema_vae.decoder.forward_manual_latents(len(samp[0]), samp, t=None)
    sample_from_latent = ema_vae.decoder.out_net.sample(px_z)

    for j in range(len(sample_from_latent)):
        im = sample_from_latent[j]
        im = Image.fromarray(im)
        im = im.resize((512, 512), resample=3)
        im.save('{}/{}.png'.format(save_directory, i * batch_size + j))

  0%|          | 0/33 [00:00<?, ?it/s]

0
30
60
90
120
150
180
210
240
270
300
330
360
390
420
450
480
510
540
570
600
630
660
690
720
750
780
810
840
870
900
930
960
